In [32]:
import h5py, os
import numpy as np

In [33]:
f = h5py.File(os.getcwd()+'/data/lhe_data.h5', 'r+')
#f = h5py.File(os.getcwd()+'/data/lhe_data_shuffled.h5', 'r+')

In [34]:
f.keys()

<KeysViewHDF5 ['benchmarks', 'morphing', 'observables', 'parameters', 'sample_summary', 'samples']>

In [35]:
bm = f['benchmarks']
bm.keys()

<KeysViewHDF5 ['is_nuisance', 'is_reference', 'names', 'values']>

In [36]:
mp = f['morphing']
mp.keys()

<KeysViewHDF5 ['components', 'morphing_matrix']>

In [37]:
ob = f['observables']
ob.keys()

<KeysViewHDF5 ['definitions', 'names']>

In [38]:
pm = f['parameters']
pm.keys()

<KeysViewHDF5 ['lha_blocks', 'lha_ids', 'max_power', 'names', 'ranges', 'transforms']>

In [39]:
ss = f['sample_summary']
ss.keys()

<KeysViewHDF5 ['background_events', 'signal_events_per_benchmark']>

In [40]:
samples = f['samples']
samples.keys()

<KeysViewHDF5 ['observations', 'sampling_benchmarks', 'weights']>

In [41]:
np.array(f['samples/sampling_benchmarks'])

array([3., 3., 3., ..., 9., 9., 9.])

In [11]:
f.close()

### Samples

In [15]:
observations = np.array(samples['observations'])
observations.shape

(6000000, 9)

In [42]:
weights = np.array(samples['weights'])
weights

array([[1.00000000e+10, 1.00000000e+10, 1.00000000e+10, ...,
        1.00000000e+10, 1.00000000e+10, 1.00000000e+10],
       [1.00000000e+10, 1.00000000e+10, 1.00000000e+10, ...,
        1.00000000e+10, 1.00000000e+10, 1.00000000e+10],
       [1.00000000e+10, 1.00000000e+10, 1.00000000e+10, ...,
        1.00000000e+10, 1.00000000e+10, 1.00000000e+10],
       ...,
       [1.00000000e+10, 1.00000000e+10, 1.00000000e+10, ...,
        1.00000000e+10, 1.00000000e+10, 2.08103695e-12],
       [1.00000000e+10, 1.00000000e+10, 1.00000000e+10, ...,
        1.00000000e+10, 1.00000000e+10, 2.08103695e-12],
       [1.00000000e+10, 1.00000000e+10, 1.00000000e+10, ...,
        1.00000000e+10, 1.00000000e+10, 2.08103695e-12]])

In [36]:
sampling_benchmarks = np.array(samples['sampling_benchmarks'])
print('num of sm events: '+str(sum(sampling_benchmarks == 3)))
#print('num of 50 events: '+str(sum(sampling_benchmarks == 4)))
#print('num of neg_50 events: '+str(sum(sampling_benchmarks == 5)))
#print('num of 200 events: '+str(sum(sampling_benchmarks == 6)))
#print('num of neg_200 events: '+str(sum(sampling_benchmarks == 7)))
#print('num of 500 events: '+str(sum(sampling_benchmarks == 8)))
#print('num of neg_500 events: '+str(sum(sampling_benchmarks == 9)))

KeyboardInterrupt: 

### Embedding toy data

In [17]:
toydataFile = h5py.File(os.getcwd()+'/toydata/gphi_toydata.h5', 'r')

#### data

In [18]:
del f['samples/observations']
f.create_dataset('samples/observations', data=np.array(toydataFile['Data']))

<HDF5 dataset "observations": shape (6000000, 9), type "<f4">

#### sampling_benchmarks

In [19]:
N = 500000
sampling_benchmarks = np.concatenate([np.concatenate([np.ones(N)*3, np.ones(N)*i]) for i in range(4, 10)])
del f['samples/sampling_benchmarks']
f.create_dataset('samples/sampling_benchmarks', data=sampling_benchmarks)

<HDF5 dataset "sampling_benchmarks": shape (6000000,), type "<f8">

#### weights

In [21]:
Weights = np.array(toydataFile['Weights'])

In [22]:
Weights

array([1.0429338e-12, 1.0429338e-12, 1.0429338e-12, ..., 2.8697630e-12,
       6.3399472e-12, 4.3035228e-10], dtype=float32)

In [24]:
weights = np.ones([N*12, 10])*(1e10)

In [25]:
#weights[N*0:N*1, 3] = Weights[N*0]
#weights[N*1:N*2, 4] = Weights[N*1]
#weights[N*:N*1, 3] = Weights[N*0]

In [26]:
for i in range(12):
    if i%2==0:
        print('sm data, pos=%d'%(3))
        weights[N*i:N*(i+1), 3] = Weights[N*i]
    else:
        print('bsm data, pos=%d'%(3+(i+1)/2))
        weights[N*i:N*(i+1), int(3+(i+1)/2)] = Weights[N*i]

sm data, pos=3
bsm data, pos=4
sm data, pos=3
bsm data, pos=5
sm data, pos=3
bsm data, pos=6
sm data, pos=3
bsm data, pos=7
sm data, pos=3
bsm data, pos=8
sm data, pos=3
bsm data, pos=9


In [27]:
del f['samples/weights']
f.create_dataset('samples/weights', data=weights)

<HDF5 dataset "weights": shape (6000000, 10), type "<f8">

#### observable numbers

In [28]:
f['observables'].keys()

<KeysViewHDF5 ['definitions', 'names']>

In [29]:
print(np.array(f['observables/definitions']))
print(np.array(f['observables/names']))

[b's' b'theta' b'thetaZ' b'thetaW' b'Sin(phiZ)' b'Sin(phiW)' b'Cos(phiZ)'
 b'Cos(phiW)' b'Pt']
[b's' b'theta' b'thetaZ' b'thetaW' b'Sin(phiZ)' b'Sin(phiW)' b'Cos(phiZ)'
 b'Cos(phiW)' b'Pt']


In [30]:
del f['observables/definitions']
del f['observables/names']

observables = ['s', 'theta', 'thetaZ', 'thetaW', 'Sin(phiZ)', 'Sin(phiW)', 'Cos(phiZ)', 'Cos(phiW)', 'Pt']
f.create_dataset('observables/definitions', data=(np.array(observables, dtype='S')))
f.create_dataset('observables/names', data=(np.array(observables, dtype='S')))

<HDF5 dataset "names": shape (9,), type "|S9">

In [31]:
f.close()